In [8]:
from google.colab import drive
drive.mount('/transaction_detail.json')

Mounted at /transaction_detail.json


In [10]:
!ls /transaction_detail.json

MyDrive


In [11]:
pip install mstarpy

In [12]:
import json
from google.colab import files
import pprint

# Upload the file
uploaded = files.upload()

# Get the uploaded filename from the 'uploaded' dictionary
json_file = next(iter(uploaded))

# Load the JSON data
with open(json_file, 'r') as file:
    data = json.load(file)

# Print the keys of the JSON to explore its structure
print("Keys in the JSON file:", data.keys())

# Optionally, print a portion of the data to further explore its structure
pprint.pprint(data)


Streaming output truncated to the last 5000 lines.
                              'stampDuty': '0.15',
                              'sttTax': '0.00',
                              'tax': '0.00',
                              'totalTax': '0.00',
                              'trxnAmount': '2999.85',
                              'trxnCharge': '0.00',
                              'trxnDate': '27-FEB-2023',
                              'trxnDesc': 'SIP Purchase Instalment No - 1 - '
                                          'INZ000031633',
                              'trxnMode': 'N',
                              'trxnTypeFlag': 'APS',
                              'trxnUnits': '20.253'},
                             {'amc': 'P',
                              'amcName': 'ICICI Prudential Mutual Fund',
                              'checkDigit': '93',
                              'email': 'dummy@gmail.com',
                              'folio': '21407675',
                           

In [13]:
# Main Code
import json
from datetime import datetime, timedelta
from collections import defaultdict
import mstarpy


In [14]:
# Specify the JSON file location
json_file = "transaction_detail.json"

# Load the JSON data
with open(json_file, 'r') as file:
    data = json.load(file)



In [20]:
# function to fetch current NAV using mstarpy
def get_current_nav(isin):
    try:
        fund = mstarpy.Funds(term=isin, country="in")
        end_date = datetime.now()
        start_date = end_date - timedelta(days=365)
        history = fund.nav(start_date=start_date, end_date=end_date, frequency="daily")

        # Return the most recent NAV
        if history and len(history) > 0:
            return history[-1]['nav']
        return None
    except Exception as e:
        print(f"Error fetching NAV for ISIN {isin}: {e}")
        return None



In [23]:
# Process transactions based on FIFO

def print_portfolio(portfolio):
    for folio, isins in portfolio.items():
        print(f"Folio: {folio}")
        for isin, transactions in isins.items():
            print(f"  ISIN: {isin}")
            for trxn in transactions:
                print(f"    Date: {trxn['date']}, Units: {trxn['units']}, Price: {trxn['price']}, Type: {trxn['type']}, Amount: {trxn['amount']}")

# After processing the portfolio
print("Portfolio after processing transactions:")
print_portfolio(portfolio)


Portfolio after processing transactions:
Folio: 1039101537
  ISIN: INF209K01UN8
    Date: 2019-08-13 00:00:00, Units: 32.723, Price: 30.56, Type: purchase, Amount: 1000.0
    Date: 2019-08-14 00:00:00, Units: 0.0, Price: 0.0, Type: address updated from kra data, Amount: 0.0
    Date: 2020-01-20 00:00:00, Units: 258.324, Price: 34.84, Type: purchase, Amount: 9000.0
    Date: 2024-07-04 00:00:00, Units: 0.0, Price: 0.0, Type: registration of nomineenominee addition through - mfc-515661304-323945852, Amount: 0.0
Folio: 22399192
  ISIN: INF090I01JR0
    Date: 2020-02-14 00:00:00, Units: 121.978, Price: 40.991, Type: purchase, Amount: 5000.0
    Date: 2020-03-02 00:00:00, Units: 26.073, Price: 38.3535, Type: purchase, Amount: 1000.0
    Date: 2020-04-27 00:00:00, Units: -148.051, Price: 39.0157, Type: redemption of units, Amount: -5776.31
Folio: 2684715
  ISIN: INF194K01Y29
    Date: 2019-08-13 00:00:00, Units: 18.355, Price: 54.48, Type: purchase us 80c of it act 1961, Amount: 1000.0
    D

In [24]:
# Calculate total portfolio value
def calculate_portfolio_value(portfolio):
    total_value = 0
    portfolio_value_by_scheme = {}

    for folio, schemes in portfolio.items():
        for isin, transactions in schemes.items():
            current_nav = get_current_nav(isin)
            if current_nav is None:
                continue

            remaining_units = 0
            for trxn in transactions:
                if trxn['type'] == 'purchase':
                    remaining_units += trxn['units']
                elif trxn['type'] == 'sell':
                    remaining_units -= trxn['units']

            current_value = remaining_units * current_nav
            portfolio_value_by_scheme[isin] = current_value
            total_value += current_value

    return total_value, portfolio_value_by_scheme

# Calculate and print total portfolio value
total_value, _ = calculate_portfolio_value(portfolio)
print("Total Portfolio Value: ₹{:.2f}".format(total_value))


Total Portfolio Value: ₹1535475.69


In [22]:
# Calculate portfolio gain
def calculate_portfolio_gain(portfolio):
    total_gain = 0
    portfolio_gain_by_scheme = {}

    for folio, schemes in portfolio.items():
        for isin, transactions in schemes.items():
            current_nav = get_current_nav(isin)
            if current_nav is None:
                continue

            total_units = 0
            acquisition_cost = 0

            for trxn in transactions:
                if trxn['type'] == 'purchase':
                    total_units += trxn['units']
                    acquisition_cost += trxn['units'] * trxn['price']

            current_value = total_units * current_nav
            gain = current_value - acquisition_cost
            portfolio_gain_by_scheme[isin] = gain
            total_gain += gain

    return total_gain, portfolio_gain_by_scheme

# Calculate and print total portfolio gain
total_gain, _ = calculate_portfolio_gain(portfolio)  # Ignore portfolio_gain_by_scheme
print("\nTotal Portfolio Gain: ₹{:.2f}".format(total_gain))



Total Portfolio Gain: ₹552519.21
